## HOME WORK-02
NAME :Bhuma Poorna Sasi Lakshmi Sai Sree


Question-01

Now we import some of the libraries usually needed by our workload.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [2]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()


Data Preprocessing
In this step, rather than downloading a file from Google Drive, we will load a famous machine learning dataset, the Breast Cancer Wisconsin dataset, using the scikit-learn datasets loader.

For convenience, given that the dataset is small, we first

construct a Pandas dataframe
tune the schema
and convert it into a Spark dataframe.

In [3]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
pd_df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df = spark.createDataFrame(pd_df)
pd_df.head()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
df.printSchema()

root
 |-- mean radius: double (nullable = true)
 |-- mean texture: double (nullable = true)
 |-- mean perimeter: double (nullable = true)
 |-- mean area: double (nullable = true)
 |-- mean smoothness: double (nullable = true)
 |-- mean compactness: double (nullable = true)
 |-- mean concavity: double (nullable = true)
 |-- mean concave points: double (nullable = true)
 |-- mean symmetry: double (nullable = true)
 |-- mean fractal dimension: double (nullable = true)
 |-- radius error: double (nullable = true)
 |-- texture error: double (nullable = true)
 |-- perimeter error: double (nullable = true)
 |-- area error: double (nullable = true)
 |-- smoothness error: double (nullable = true)
 |-- compactness error: double (nullable = true)
 |-- concavity error: double (nullable = true)
 |-- concave points error: double (nullable = true)
 |-- symmetry error: double (nullable = true)
 |-- fractal dimension error: double (nullable = true)
 |-- worst radius: double (nullable = true)
 |-- worst 

In [5]:
data_cust=df.na.drop()

In [6]:
data_cust.printSchema()

root
 |-- mean radius: double (nullable = true)
 |-- mean texture: double (nullable = true)
 |-- mean perimeter: double (nullable = true)
 |-- mean area: double (nullable = true)
 |-- mean smoothness: double (nullable = true)
 |-- mean compactness: double (nullable = true)
 |-- mean concavity: double (nullable = true)
 |-- mean concave points: double (nullable = true)
 |-- mean symmetry: double (nullable = true)
 |-- mean fractal dimension: double (nullable = true)
 |-- radius error: double (nullable = true)
 |-- texture error: double (nullable = true)
 |-- perimeter error: double (nullable = true)
 |-- area error: double (nullable = true)
 |-- smoothness error: double (nullable = true)
 |-- compactness error: double (nullable = true)
 |-- concavity error: double (nullable = true)
 |-- concave points error: double (nullable = true)
 |-- symmetry error: double (nullable = true)
 |-- fractal dimension error: double (nullable = true)
 |-- worst radius: double (nullable = true)
 |-- worst 

In [7]:
def make_df_columns_null(spark, df, column_list, nullable=False):
    
    for struct in df.schema:
        
        if struct.name in column_list:
            
            struct.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

df = make_df_columns_null(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row: Vectors.dense(row.features))

In [8]:
df.printSchema()

root
 |-- mean radius: double (nullable = false)
 |-- mean texture: double (nullable = false)
 |-- mean perimeter: double (nullable = false)
 |-- mean area: double (nullable = false)
 |-- mean smoothness: double (nullable = false)
 |-- mean compactness: double (nullable = false)
 |-- mean concavity: double (nullable = false)
 |-- mean concave points: double (nullable = false)
 |-- mean symmetry: double (nullable = false)
 |-- mean fractal dimension: double (nullable = false)
 |-- radius error: double (nullable = false)
 |-- texture error: double (nullable = false)
 |-- perimeter error: double (nullable = false)
 |-- area error: double (nullable = false)
 |-- smoothness error: double (nullable = false)
 |-- compactness error: double (nullable = false)
 |-- concavity error: double (nullable = false)
 |-- concave points error: double (nullable = false)
 |-- symmetry error: double (nullable = false)
 |-- fractal dimension error: double (nullable = false)
 |-- worst radius: double (nullable

With the next cell, we build the two data structures that we will be using throughout this file:

features, a dataframe of Dense vectors, containing all the original features in the dataset;
labels, a series of binary labels indicating if the corresponding set of features belongs to a subject with breast cancer, or not.

In [9]:
from pyspark.ml.linalg import Vectors
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = pd.Series(breast_cancer.target)

## Your task
If you run successfully the Setup and Data Preprocessing stages, you are now ready to cluster the data with the K-means algorithm included in MLlib (Spark's Machine Learning library). Set the k parameter to 2, fit the model, and the compute the Silhouette score (i.e., a measure of quality of the obtained clustering).

IMPORTANT: use the MLlib implementation of the Silhouette score (via ClusteringEvaluator).

In [10]:
from pyspark.ml.clustering import KMeans

In [11]:
from pyspark.ml.evaluation import ClusteringEvaluator
# Trains a k-means model.
kmeans_model = KMeans().setK(2).setSeed(1)
model_fit = kmeans_model.fit(features)

In [12]:
# Make predictions
predictions_kmeans = model_fit.transform(features)
# Evaluate clustering by computing Silhouette score
evaluator_kmeans = ClusteringEvaluator()
silhouette_kmeans = evaluator_kmeans.evaluate(predictions_kmeans)

In [13]:
print(f'Silhouette: {silhouette_kmeans}')

Silhouette: 0.8342904262826143


Take the predictions produced by K-means, and compare them with the labels variable (i.e., the ground truth from our dataset).

Compute how many data points in the dataset have been clustered correctly (i.e., positive cases in one cluster, negative cases in the other).

HINT: you can use np.count_nonzero(series_a == series_b) to quickly compute the element-wise comparison of two series.

IMPORTANT: K-means is a clustering algorithm, so it will not output a label for each data point, but just a cluster identifier! As such, label 0 does not necessarily match the cluster identifier 0.

In [15]:
predictions_needed = predictions_kmeans.toPandas()
converted_result = predictions_needed['prediction'].apply(lambda x: 0 if x else 1)
np.count_nonzero(converted_result.values == labels.values)

486

Now perform dimensionality reduction on the features using the PCA statistical procedure, available as well in MLlib.

Set the k parameter to 2, effectively reducing the dataset size of a 15X factor.

In [16]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
pca_model = PCA(k=2, inputCol="features", outputCol="pca")
model_pca = pca_model.fit(features)
result = model_pca.transform(features).select("pca")
result.show(truncate=False)

+----------------------------------------+
|pca                                     |
+----------------------------------------+
|[-2260.013886292541,-187.96030122263645]|
|[-2368.9937557820544,121.5874242581556] |
|[-2095.665201547861,145.11398565870158] |
|[-692.6905100570509,38.57692259208186]  |
|[-2030.2124927427062,295.2979839927929] |
|[-888.2800535760762,26.079796157025843] |
|[-1921.0822124748452,58.80757247309972] |
|[-1074.7813350047961,31.771227808469845]|
|[-908.5784781618831,63.83075279044658]  |
|[-861.578449407568,40.57073549705342]   |
|[-1404.5591306499473,88.2321825773626]  |
|[-1524.2324408687819,-3.263057316777707]|
|[-1734.3856477464153,273.16267815114634]|
|[-1162.9140032230357,217.63481808344636]|
|[-903.4301030498834,135.6151766608481]  |
|[-1155.875995420685,76.80889383742188]  |
|[-1335.729432130807,-2.468400545035389] |
|[-1547.264092252309,3.805675972574695]  |
|[-2714.964765181216,-164.3761086425877] |
|[-908.2502671870876,118.2164200822313]  |
+----------

Now run K-means with the same parameters as above, but on the pcaFeatures produced by the PCA reduction you just executed.

Compute the Silhouette score, as well as the number of data points that have been clustered correctly.

In [21]:
kmeans = KMeans(featuresCol='pca').setK(2).setSeed(1)
model_final = kmeans.fit(result)
pca_predictions_final = model_final.transform(result)
pca_evaluator_final = ClusteringEvaluator(featuresCol='pca')
pca_silhouette = pca_evaluator_final.evaluate(pca_predictions_final)
print(f'Silhouette of PCA {pca_silhouette}')

Silhouette of PCA 0.8348610363444832


In [22]:
pca_output = pca_predictions_final.toPandas()
converted_pca = pca_output['prediction'].apply(lambda x: 0 if x else 1)
np.count_nonzero(converted_pca == labels.values)

486

In [23]:
#stopping Spark environment
sc.stop()